# Coleta de dados

In [1]:
# - image das etapas(arquitetura) -> ok 
# - definicao das etapas do que vai ser feito e o que vai ser feito (trello) -> ok

# - descricao dos trabalhos relacionados encontrados -> ok

# - publicacoes eniac e bracis

In [2]:
import pandas as pd
import numpy as np
# import nltk
# import re
# from nltk.corpus import stopwords
# from nltk.stem import RSLPStemmer
# from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
# nltk.download('rslp')
# nltk.download('stopwords')
from  componetes_preprocessamento import RemoveStopWords, Cleaner, Tokenizador, Stemmer, Joiner, pega_resultados, salvando_em_arquivo
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\emn3\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emn3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# !conda install nltk
# import nltk
# nltk.download_gui()

In [4]:
dataset = pd.read_csv("datasets/reviews.csv")

# Pré-Processamento dos Dados

## Definindo Tipos

In [5]:
dataset.head(5)

,Unnamed: 0,order_id,review_id,review_score,review_comment_message
0,3,658677c97b385a9be170737859d3511b,e64fb393e7b32834bb789ff8bb30750e,1,Recebi bem antes do prazo estipulado.
1,4,8e6bfb81e283fa7e4f11123a3fb894f1,f7c4243c7fe1938f181bec41a392bdeb,1,Parabéns lojas lannister adorei comprar pela I...
2,9,b9bf720beb4ab3728760088589c62129,8670d52e15e00043ae7de4c01cc2fe06,0,aparelho eficiente. no site a marca do aparelh...
3,12,9d6f15f95d01e79bd1349cc208361f09,4b49719c8a200003f700d3d986ea1a19,0,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,15,e51478e7e277a83743b6f9991dbfa3fb,3948b09f7c818e2d86c9a546758b2335,1,"Vendedor confiável, produto ok e entrega antes..."


In [6]:
X = dataset["review_comment_message"].copy()
y = dataset["review_score"].copy()
y = np.array(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 199)

# Resultados

In [8]:
resultados = []

### Accuracy - GridSearchCV

In [9]:
modelo = MultinomialNB()
parametros = {
    "fit_prior": [False, True],
    "alpha": np.arange(.0, 1.1, .01)
}
kfold = StratifiedKFold(10,shuffle=True, random_state=1)
acc_gs_pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("MNB", GridSearchCV(modelo, parametros, scoring="accuracy", cv=kfold, n_jobs=-1)),
                    ])
acc_gs_pipeline.fit(X_train, y_train)

Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('MNB',
                 GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
                              estimator=MultinomialNB(), n_jobs=-1,
                              param_grid={'alpha':...
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09]),
                               

In [10]:
cvres = acc_gs_pipeline["MNB"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args[:5]:
    print(cvres["mean_test_score"][i], cvres["params"][i])

0.7253155706602517 {'alpha': 1.08, 'fit_prior': True}
0.7252806178465502 {'alpha': 1.09, 'fit_prior': True}
0.725280605633827 {'alpha': 1.07, 'fit_prior': True}
0.725280605633827 {'alpha': 1.06, 'fit_prior': True}
0.7252107122191471 {'alpha': 0.92, 'fit_prior': True}


In [11]:
y_pred = acc_gs_pipeline.predict(X_test)
resultado = pega_resultados("mnb", "Grid Search", y_test, y_pred, "acurracia", acc_gs_pipeline["MNB"].best_params_)
resultados.append(resultado)
print(resultado)

['mnb', 'Grid Search', 0.7299192693468156, 0.6581404909642637, 0.6819187841228379, 0.6624766255660014, 'acurracia', {'alpha': 1.08, 'fit_prior': True}]


### F1 Score - GridSearchCV

In [12]:
modelo = MultinomialNB()
parametros = {
    "fit_prior": [False, True],
    "alpha": np.arange(.0, 1.1, .01)
}
kfold = StratifiedKFold(10)
f1_gs_pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("MNB", GridSearchCV(modelo, parametros, scoring="f1_macro", cv=kfold, n_jobs=-1)),
                    ])
f1_gs_pipeline.fit(X_train, y_train)

Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('MNB',
                 GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                              estimator=MultinomialNB(), n_jobs=-1,
                              param_grid={'alp...
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09]),
                               

In [13]:
cvres = f1_gs_pipeline["MNB"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args[:5]:
    print(cvres["mean_test_score"][i], cvres["params"][i])

0.6623775620930415 {'alpha': 0.28, 'fit_prior': False}
0.662320223798129 {'alpha': 0.29, 'fit_prior': False}
0.6622918138935106 {'alpha': 0.27, 'fit_prior': False}
0.6622534928463722 {'alpha': 0.26, 'fit_prior': False}
0.6622491866253623 {'alpha': 0.31, 'fit_prior': False}


In [14]:
y_pred = f1_gs_pipeline.predict(X_test)
resultado = pega_resultados("mnb", "Grid Search", y_test, y_pred, "f1 score", f1_gs_pipeline["MNB"].best_params_)
resultados.append(resultado)
print(resultado)

['mnb', 'Grid Search', 0.716464160482753, 0.6628304413922802, 0.6661534839172297, 0.6657296070451265, 'f1 score', {'alpha': 0.28, 'fit_prior': False}]


### Accuracy - RandomizedSearchCV

In [15]:
modelo = MultinomialNB()
parametros = {
    "fit_prior": [False, True],
    "alpha": np.arange(.0, 1.1, .01)
}
kfold = StratifiedKFold(10)
acc_rs_pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("MNB", RandomizedSearchCV(modelo, parametros, scoring="f1_macro", cv=kfold, n_jobs=-1)),
                    ])
acc_rs_pipeline.fit(X_train, y_train)

Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('MNB',
                 RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                                    estimator=MultinomialNB(), n_jobs=-1,
                                    param_dist...
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09]),
                   

In [16]:
cvres = acc_rs_pipeline["MNB"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args[:5]:
    print(cvres["mean_test_score"][i], cvres["params"][i])

0.6617666600406571 {'fit_prior': False, 'alpha': 0.43}
0.6615536904963093 {'fit_prior': False, 'alpha': 0.63}
0.6615509843052335 {'fit_prior': False, 'alpha': 1.05}
0.6614271534012616 {'fit_prior': False, 'alpha': 0.51}
0.6613371572864596 {'fit_prior': False, 'alpha': 1.04}


In [17]:
y_pred = acc_rs_pipeline.predict(X_test)
resultado = pega_resultados("mnb", "Randomized Search", y_test, y_pred, "acuracia", acc_rs_pipeline["MNB"].best_params_)
resultados.append(resultado)
print(resultado)

['mnb', 'Randomized Search', 0.7178504444263231, 0.6637466424296652, 0.6674642642913321, 0.6668492928324167, 'acuracia', {'fit_prior': False, 'alpha': 0.43}]


### F1 Score - RandomizedSearchCV

In [18]:
modelo = MultinomialNB()
parametros = {
    "fit_prior": [False, True],
    "alpha": np.arange(.0, 1.1, .01)
}
kfold = StratifiedKFold(10)
f1_rs_pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("MNB", RandomizedSearchCV(modelo, parametros, scoring="f1_macro", cv=kfold, n_jobs=-1)),
                    ])
f1_rs_pipeline.fit(X_train, y_train)

Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('MNB',
                 RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                                    estimator=MultinomialNB(), n_jobs=-1,
                                    param_dist...
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09]),
                   

In [19]:
cvres = f1_rs_pipeline["MNB"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args[:5]:
    print(cvres["mean_test_score"][i], cvres["params"][i])

0.6621554852065702 {'fit_prior': False, 'alpha': 0.21}
0.6618319649715014 {'fit_prior': False, 'alpha': 0.77}
0.6617660023170305 {'fit_prior': False, 'alpha': 0.38}
0.6614447024191229 {'fit_prior': False, 'alpha': 0.5700000000000001}
0.6613371572864596 {'fit_prior': False, 'alpha': 1.04}


In [20]:
y_pred = f1_rs_pipeline.predict(X_test)
resultado = pega_resultados("mnb", "Randomized Search", y_test, y_pred, "f1 score", acc_rs_pipeline["MNB"].best_params_)
resultados.append(resultado)
print(resultado)

['mnb', 'Randomized Search', 0.7160564299111147, 0.6625124357075344, 0.6656240803683947, 0.665389826962964, 'f1 score', {'fit_prior': False, 'alpha': 0.43}]


In [22]:
salvando_em_arquivo("resultados/MNB_resultados.csv", resultados)